Notes:

Section 17.2 - MLE for Bayesian Networks

Ideas to mention
- The likelihood function - Parameters impact the chain rule, that tells us how to calc probabilities. We maximize those probabilities for our data
- The decomposition of the likelihood
- Bayesian parameter estimation.
- Prediction? Integrating the parameters out.
- What happens with missing data?

Answer structure:

- Intro
    -  
- Refresher
    - 
- Where are we starting and what's the goal?
    - 

### How are the parameters of a Bayesian Network learned?

(This is the 5th answer in a 7 part series [link] on Probabilistic Graphical Models ('PGMs'). At the least, I suggest checking out the notation guide from that linked 1st answer.)

Up until now, our starting point is that we've been given a PGM (of which a Bayesian Network (a 'BN') is an certain type), complete with fitted parameters. Here, we'll discover how to determine (i.e. to 'learn') those parameters from data. I'll say up front, this task for a BN is as friendly as parameter learning gets - that's why its our first destination. In doing so, we'll take a tour of the major considerations of parameter learning and their relatively simple solutions for a BN. When we're faced with more complicated parameter learning tasks, the approach here will serve as a useful framework to contextualize those difficulties.

As promised, let's recap some relevant information.

### Refresher

In the first answer [link], we discovered why PGMs are useful tools for representing complex system. We defined a complex system as a set of $n$ random variables (which we call $\mathcal{X}$) with a relationship we'd like to understand. We take it that there exists some true but unknown joint distribution, $P$, which govern these random variables ('RVs'). We take it that a 'good understanding' means we can answer two types of questions regarding this $P$:

1. **Probability Queries**: Compute the probabilities $P(\mathbf{Y}|\mathbf{e})$. What is the distribution of the RV's of $\mathbf{Y}$ given we have some observation ($\mathbf{e}$) of the RVs of $\mathbf{E}$?
2. **MAP Queries**: Determine $\textrm{argmax}_\mathbf{Y}P(\mathbf{Y}|\mathbf{e})$. That is, determine the most likely values of some RVs given an assignment of other RVs.

(Where $\mathbf{E}$ and $\mathbf{Y}$ are two arbitrary subsets of $\mathcal{X}$. If this notation is unfamiliar, see the 'Notation Guide' section from the first answer [link]).

The idea behind PGMs is to estimate $P$ using two things:

1. A graph: a set of nodes, each of which represents an RV from $\mathcal{X}$, and a set of edges between these nodes.
2. Parameters: objects that, when paired with a graph and a certain rule, allow us to calculate probabilities of assignments of $\mathcal{X}$.

One of the two main categories of PGM is the Bayesian Networks. A **Bayesian Network** involves a graph, denoted as $\mathcal{G}$, with *directed* edges and no directed cycles. The parameters are Conditional Probability Tables ('CPDs' or 'CPTs'), which are, as the naming suggests, select conditional probabilities from the BN. They give us the right hand side of the Chain Rule, which dictates we calculate probabilities according to a BN:

$$
P_{B}(X_1,\cdots,X_n)=\prod_{i=1}^n P_{B}(X_i|\textrm{Pa}_{X_i}^\mathcal{G})
$$

To illustrate, we can consider a well utilized example from the text Probabilistic Graphical Models (link). It's called the 'Student Bayesian Network'. Here, we're concerned with a system of five RVs: a student's intelligence ($I$), their class's difficulty ($D$), their grade in that class ($G$), their letter of recommendation ($L$) and their SAT score ($S$). So $\mathcal{X}=\{I,D,G,L,S\}$. The BN graph along with the CPDs can be represented as:

![title](StudentBN.png)

According to our rule, we have that any joint assignment of $\mathcal{X}$ factors as:

$$
P_B(I,D,G,S,L) = P_B(I)P_B(D)P_B(G|I,D)P_B(S|I)P_B(L|G)
$$

So we would calculate a given assignment as:

$$
\begin{align}
P_B(i^1,d^0,g^2,s^1,l^0) = & P_B(i^1)P_B(d^0)P_B(g^2|i^1,d^0)P_B(s^1|i^1)P_B(l^0|g^2)\\
= & 0.3\cdot 0.6\cdot 0.08\cdot 0.8\cdot0.4 \\
= & 0.004608\\
\end{align}
$$

Now, everything we've discussed so far has assumed those tables are *given* to us. Not anymore!

### What's our starting point and what's our goal?

Our job is to determine the CPTs associated with the *given* graph of a BN. We are assisted by the fact that we have some, at least partial, observations of $\mathcal{X}$. For the first part of this answer, we'll assume our observations of $\mathcal{X}$ are *complete* - each piece of data is a observation of an assignment to *all* RVs of $\mathcal{X}$. Later, we'll relax this assumption and consequently, ruin our lives. 

But, with our 